### tf > keras 실험 코드

In [2]:
import tensorflow as tf
import numpy as np
tf.__version__

'2.1.0'

In [3]:
img_shapes = [256,256,3]

In [3]:
with open('data/places2/train_shuffled.flist') as f:
    fnames = f.read().splitlines()
print(fnames[0])

/Users/singwanghyeon/PycharmProjects/tfcoreml/generative_inpainting-master/examples/makepb/training/training/k.png


In [ ]:
data = ng.data.DataFromFNames(
        fnames, img_shapes, random_crop=FLAGS.random_crop,
        nthreads=FLAGS.num_cpus_per_job)
images = data.data_pipeline(FLAGS.batch_size)

In [ ]:
# Queue 설명
# https://bcho.tistory.com/1163
# 2.0에서는 tf.data를위해 Queue Runner 제거됨

In [ ]:
# 큐 사이즈 256
from tensorflow.python.ops import data_flow_ops
queue = data_flow_ops.FIFOQueue(
    capacity=256,
    dtypes=tf.float32,
    shapes=img_shapes,
    shared_name=None,
    name=None)
batch_size=16
data = queue.dequeue_many(batch_size)

In [4]:
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)